# VGG-16 on CIFAR-10
This notebook is used to experiment with VGG-16 on CIFAR-10 dataset.

## Setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import sys

sys.path.append('../../')
sys.path.append('../../src/')

import src.general as general
import src.dataset_models as data
import src.metrics as metrics
import src.evaluation as eval
import src.plot as plot
import src.compression.distillation as distill
import src.compression.pruning as prune


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Get model
vgg_weights = tv.models.VGG16_Weights.DEFAULT
vgg16 = tv.models.vgg16(weights=vgg_weights)

# Modify the last layer to have 10 output classes (CIFAR-10 has 10 classes)
vgg16.classifier[6] = nn.Linear(4096, 10)


In [5]:
# Get dataset
dataset = data.supported_datasets["CIFAR-10"]

In [6]:
# Get transforms
vgg_cifar10_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Set transforms
dataset.set_transforms(vgg_cifar10_transform)

In [7]:
general.get_device()

device(type='cuda')

In [26]:
pruned_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [27]:
after_results = general.test(pruned_model, dataset)

Test: 100%|██████████| 157/157 [00:26<00:00,  5.94it/s]

Test loss: 0.2095
Test score: 93.8893


In [10]:
general.finetune(vgg16, dataset, target=99, max_it=5)

Train: 100%|██████████| 782/782 [05:11<00:00,  2.51it/s]


Train loss: 0.1157
Train score: 95.9639


Train: 100%|██████████| 782/782 [05:11<00:00,  2.51it/s]


Train loss: 0.0921
Train score: 96.8151


Train: 100%|██████████| 782/782 [05:11<00:00,  2.51it/s]


Train loss: 0.0776
Train score: 97.2946


Train: 100%|██████████| 782/782 [05:11<00:00,  2.51it/s]


Train loss: 0.0639
Train score: 97.7522


Train: 100%|██████████| 782/782 [05:10<00:00,  2.52it/s]

Train loss: 0.0566
Train score: 98.1178
Maximum number of iterations reached
Finetuning finished after 5 iterations


In [19]:
torch.save(vgg16.state_dict(), "/workspace/volume/models/vgg16_cifar10_pruned_50.pt")

In [42]:
eval.get_size(vgg16)

512.33

In [44]:
general.test(vgg16, dataset)

Test:   0%|          | 0/157 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 9.78 GiB total capacity; 4.85 GiB already allocated; 3.12 MiB free; 6.32 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [28]:
before_results = eval.get_results(vgg16, dataset)

Test: 100%|██████████| 157/157 [00:26<00:00,  6.02it/s]


Test loss: 0.2073
Test score: 93.8595
Could not calculate FLOPS


In [37]:
ignored_layers = prune.get_layers_not_to_prune(vgg16.features)

In [40]:
list(vgg16.children())

[Sequential(
   (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU(inplace=True)
   (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU(inplace=True)
   (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (6): ReLU(inplace=True)
   (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (8): ReLU(inplace=True)
   (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (11): ReLU(inplace=True)
   (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (13): ReLU(inplace=True)
   (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (15): ReLU(inplace=True)
   (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (17): Conv2d(

In [38]:
ignored_layers

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]

In [41]:
pruned_model = prune.magnitude_pruning_structured(vgg16, dataset, 0.5, iterative_steps=5)

OutOfMemoryError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 9.78 GiB total capacity; 4.78 GiB already allocated; 117.12 MiB free; 6.32 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [30]:
after_results = eval.get_results(pruned_model, dataset)

Test: 100%|██████████| 157/157 [00:25<00:00,  6.04it/s]


Test loss: 0.2122
Test score: 93.6306
Could not calculate FLOPS


In [31]:
plot.print_before_after_results(before_results, after_results)

============================= RESULTS BEFORE & AFTER ===========================
Loss: 0.207255 -> 0.212175 (2.37%)
Score: 93.859475 -> 93.630573 (-0.24%)
Time per data point: 10.3823 ms -> 10.3398 ms (-0.41%)
Model Size: 512.33 MB -> 512.33 MB (-0.00%)
Number of parameters: 134301514 -> 134301514 (-0.00%)
Number of MACs: 15499459200 -> 15499459200 (-0.00%)
